In [1]:
import os
import time
import re
import random
import requests
import random
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

In [2]:
import os
import time
import re
import random
import requests
import random
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup


### Các bước chuẩn bị
# Function to read proxy config from file
def read_proxy_config(file_path):
    config = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            config[key] = value
    return config

# Read proxy settings from text file
proxy_config = read_proxy_config(r'C:\working\job_rcm\job_rcm_code\config.txt')
PROXY_HOST = proxy_config['PROXY_HOST']
PROXY_PORT = proxy_config['PROXY_PORT']
PROXY_USER = proxy_config['PROXY_USER']
PROXY_PASS = proxy_config['PROXY_PASS']

# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Manifest for proxy
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

# Background script for proxy
background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
	
# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)


# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("http://www.facebook.com")

# Login 
time.sleep(4)
email = driver.find_element(By.NAME, "email")
password = driver.find_element(By.NAME, "pass")
email.send_keys("nguyenanhnguyen111666@gmail.com")
password.send_keys("tranduyluan11062003")

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# Open a new tab
driver.execute_script("window.open('https://www.facebook.com/groups/datajobvn/');")
driver.switch_to.window(driver.window_handles[-1])
# Scale screen to 60% for the new tab as well
driver.execute_script("document.body.style.zoom='60%'")

C:\Users\trand\AppData\Local\Temp\ipykernel_3620\2925264960.py:121: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


In [ ]:
import os
import time
import requests
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

# Set up directories
image_save_dir = r'C:\working\job_rcm\data\facebook\post_image'
csv_file_path = r'C:\working\job_rcm\data\facebook\post.csv'

if not os.path.exists(image_save_dir):
    os.makedirs(image_save_dir)

# Initialize lists for data
post_href = []
hr_id = []  # Fill with "chưa lấy"
group_id = []  # Fill with "group 1"
scrape_date = []
author_name = []
content = []
image = []
post_date = []

actions = ActionChains(driver)

while True:
    # Find the feed element
    feed_element = driver.find_element(By.XPATH, '//*[@role="feed"]')
    post_elements = driver.find_elements(By.XPATH, "//div[@class='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z']")
    
    for index, element in enumerate(post_elements):
					# Scroll to the current element
					actions.move_to_element(element).perform()
					time.sleep(5)  # Wait for more posts to load
					
					# Get post href
					post_href_element = element.find_element(By.XPATH, ".//a[@role='link']")
					post_href_value = post_href_element.get_attribute('href')

					# Skip if post_href is already in the list
					if post_href_value in post_href:
							continue
					
					# Get author name
					author_name_element = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
					author_name_value = author_name_element[0].text.strip()
					
					actions.move_to_element(author_name_element[0]).key_down(Keys.CONTROL).click(author_name_element[0]).key_up(Keys.CONTROL).perform()
					
					# Chờ cho tab mới mở ra
					time.sleep(3)
					# Chuyển sang tab mới
					# Lấy danh sách các tab hiện tại
					current_tabs = driver.window_handles

					# Kiểm tra nếu có ít nhất 2 tab
					if len(current_tabs) > 2:
							# Chuyển sang tab mới (tab cuối cùng trong danh sách tab)
							driver.switch_to.window(current_tabs[-1])

							# Tìm href của hr và điền vào hr_id
							profile_elements = driver.find_elements(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx')
							hr_id_value = profile_elements[6].get_attribute('href')

							time.sleep(2)

							# Đóng tab mới nếu tồn tại tab thứ 2
							driver.close()

							# Chuyển về tab chính (tab đầu tiên trong danh sách tab)
							driver.switch_to.window(current_tabs[1])
					else:
							print("Chỉ có một tab, không có tab nào để đóng.")

					time.sleep(5)

					group_id_value = "group 1"
					scrape_date_value = time.strftime('%Y-%m-%d %H:%M:%S')
					
					
					####### Get post date with try-except block
					try:
							hover_elements = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
							time.sleep(5)
							actions.move_to_element(hover_elements[1]).perform()
							time.sleep(5)
							# date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xlh3980.xvmahel.x1n0sxbx.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
							date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
							post_date_value = date_element.text
							print(post_date_value)
					except Exception as e:
							print(f"Không lấy ngày đăng bài ở {index+1}")
							post_date_value = None
					
					####### Check for "see more" button and click it if present
					try:
							see_more_button = element.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl') and contains(@role, 'button') and contains(text(), 'Xem thêm')]")
							if see_more_button:
									see_more_button.click()
									time.sleep(1)  # Wait for content to expand
					except Exception as e:
							print("Post không có nút xem thêm")

					# Get the content
					content_element_1s = element.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")

					if len(content_element_1s) > 1:
							print('1')
							content_value = content_element_1s[1].text
					else: 
							time.sleep(1)
							print('2')
							content_element_2s = element.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u")

							if content_element_2s:
									content_value = content_element_2s[0].text
							else:
									content_value = None
									print("No content found for this post")
					
					# Get images
					image_elements = element.find_elements(By.TAG_NAME, 'img')
					image_urls = [img.get_attribute('src') for img in image_elements if int(img.get_attribute('width')) > 100 and int(img.get_attribute('height')) > 100]
					if image_urls:
							post_image_dir = os.path.join(image_save_dir, f'post_{index+1}')
							if not os.path.exists(post_image_dir):
									os.makedirs(post_image_dir)
							
							for i, image_url in enumerate(image_urls, start=1):
									image_path = os.path.join(post_image_dir, f'image_{i}.jpg')
									try:
											image_data = requests.get(image_url).content
											with open(image_path, 'wb') as img_file:
													img_file.write(image_data)
									except Exception as e:
											print(f"post {index+1} không có ảnh")
							image_value = ", ".join(image_urls)
					else:
							image_value = None

					# Append data to lists
					post_href.append(post_href_value)
					hr_id.append(hr_id_value)
					author_name.append(author_name_value)
					group_id.append(group_id_value)
					scrape_date.append(scrape_date_value)
					post_date.append(post_date_value)
					content.append(content_value)
					image.append(image_value)
					
					# Create DataFrame and save to CSV after each iteration
					df = pd.DataFrame({
							'post_href': post_href,
							'hr_id': hr_id,
							'author_name': author_name,
							'group_id': group_id,
							'scrape_date': scrape_date,
							'post_date': post_date,
							'content': content,
							'image': image
					})
					df.to_csv(csv_file_path, index=False, encoding='utf-8')
					print(f"Lưu csv post {index+1}")


# Hoàn thiện code 

In [4]:
import os
import time
import re
import random
import requests
import random
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup


### Các bước chuẩn bị
# Function to read proxy config from file
def read_proxy_config(file_path):
    config = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            config[key] = value
    return config

# # Read proxy settings from text file
# proxy_config = read_proxy_config(r'C:\working\job_rcm\job_rcm_code\config.txt')
# PROXY_HOST = proxy_config['PROXY_HOST']
# PROXY_PORT = proxy_config['PROXY_PORT']
# PROXY_USER = proxy_config['PROXY_USER']
# PROXY_PASS = proxy_config['PROXY_PASS']

# # ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# # Manifest for proxy
# manifest_json = """
# {
#     "version": "1.0.0",
#     "manifest_version": 2,
#     "name": "Chrome Proxy",
#     "permissions": [
#         "proxy",
#         "tabs",
#         "unlimitedStorage",
#         "storage",
#         "<all_urls>",
#         "webRequest",
#         "webRequestBlocking"
#     ],
#     "background": {
#         "scripts": ["background.js"]
#     },
#     "minimum_chrome_version":"22.0.0"
# }
# """

# # Background script for proxy
# background_js = """
# var config = {
#         mode: "fixed_servers",
#         rules: {
#         singleProxy: {
#             scheme: "http",
#             host: "%s",
#             port: parseInt(%s)
#         },
#         bypassList: ["localhost"]
#         }
#     };

# chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

# function callbackFn(details) {
#     return {
#         authCredentials: {
#             username: "%s",
#             password: "%s"
#         }
#     };
# }

# chrome.webRequest.onAuthRequired.addListener(
#             callbackFn,
#             {urls: ["<all_urls>"]},
#             ['blocking']
# );
# """ % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
	
# # Set up proxy
# pluginfile = 'proxy_auth_plugin.zip'
# with zipfile.ZipFile(pluginfile, 'w') as zp:
#     zp.writestr("manifest.json", manifest_json)
#     zp.writestr("background.js", background_js)
# options.add_extension(pluginfile)


# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("http://www.facebook.com")

# Login 
time.sleep(4)
email = driver.find_element(By.NAME, "email")
password = driver.find_element(By.NAME, "pass")
email.send_keys("nguyenanhnguyen111666@gmail.com")
password.send_keys("tranduyluan11062003")

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

# # Open a new tab
# driver.execute_script("window.open('https://www.facebook.com/groups/160325109320298/');") 
# ### cần làm nhiều group trong 

# driver.switch_to.window(driver.window_handles[-1])
# # Scale screen to 60% for the new tab as well
# driver.execute_script("document.body.style.zoom='60%'")

C:\Users\trand\AppData\Local\Temp\ipykernel_17672\1808688157.py:121: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


logic code của lấy bài lấy fb 

- vòng lặp 1 : (lặp qua các group fb đã chọn, chỉ chuyển qua group mới khi đã lấy hết bài đăng của group này ) đăng tính có lẽ không cần vòng lặp 
- vòng lặp 2 nằm trong vòng lặp 1 (lấy hết bài đăng của các group, sửa logic,áp dụng index để không chạy từ trên xuống dưới)

cấu trúc thư mục sẽ là 
data > facebook > năm (attempt cào) > tháng (attempt cào) > ngày (attempt cào) > giờ phút (attempt cào)
giờ phút (attempt cào) > part (tối đa 300 thư mục 1 part, tự động tạo part mới khi đủ 300 thư mục) > 1 detail 

mỗi detail = một facebook post/tus
tên detail : [thời gian cào] _ [FbGroupName] _ [PostHref] 

1 thư mục detail gồm : file data.json , thư mục ảnh (đặt tên mỗi file ảnh trong thư mục với stt, file html.txt = outerHTML của post đó ) 



json cấu trúc của mỗi post
job i {
	post_href = 
	group_id =   # Fill with "group_href" = https://www.facebook.com/groups/160325109320298/
	scrape_date =  # còn áp dụng scrape_date cho việc tạo thư mục
	author_name = 
	content = 
	image_folder_path = 
	post_date = 
}

In [10]:
# Set up directories
image_save_dir = r'C:\working\job_rcm\data\facebook\try\post_image'
csv_file_path = r'C:\working\job_rcm\data\facebook\try\post.csv'

if not os.path.exists(image_save_dir):
    os.makedirs(image_save_dir)
	

# Initialize lists for data
post_href = []
hr_id = []  # Fill with "chưa lấy"
group_id = []  # Fill with "group 1"
scrape_date = []
author_name = []
content = []
image = []
post_date = []

actions = ActionChains(driver)

while True:
    # Find the feed element
    feed_element = driver.find_element(By.XPATH, '//*[@role="feed"]')
    post_elements = driver.find_elements(By.XPATH, "//div[@class='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z']")
    
    for index, element in enumerate(post_elements):
					# Scroll to the current element
					actions.move_to_element(element).perform()
					time.sleep(5)  # Wait for more posts to load
					
					# Get post href
					post_href_element = element.find_element(By.XPATH, ".//a[@role='link']")
					post_href_value = post_href_element.get_attribute('href')

					# Skip if post_href is already in the list
					if post_href_value in post_href:
							continue
					
					# Get author name
					author_name_element = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
					author_name_value = author_name_element[0].text.strip()
					
					actions.move_to_element(author_name_element[0]).key_down(Keys.CONTROL).click(author_name_element[0]).key_up(Keys.CONTROL).perform()
					
					# Chờ cho tab mới mở ra
					time.sleep(3)
					# Chuyển sang tab mới
					# Lấy danh sách các tab hiện tại
					current_tabs = driver.window_handles

					# Kiểm tra nếu có ít nhất 2 tab
					if len(current_tabs) > 2:
							# Chuyển sang tab mới (tab cuối cùng trong danh sách tab)
							driver.switch_to.window(current_tabs[-1])

							# Tìm href của hr và điền vào hr_id
							profile_elements = driver.find_elements(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx')
							hr_id_value = profile_elements[6].get_attribute('href')

							time.sleep(2)

							# Đóng tab mới nếu tồn tại tab thứ 2
							driver.close()

							# Chuyển về tab chính (tab đầu tiên trong danh sách tab)
							driver.switch_to.window(current_tabs[1])
					else:
							print("Chỉ có một tab, không có tab nào để đóng.")

					time.sleep(5)

					group_id_value = "group 1"
					scrape_date_value = time.strftime('%Y-%m-%d %H:%M:%S')
					
					
					####### Get post date with try-except block
					try:
							hover_elements = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
							time.sleep(5)
							actions.move_to_element(hover_elements[1]).perform()
							time.sleep(5)
							# date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.xlh3980.xvmahel.x1n0sxbx.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
							date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
							post_date_value = date_element.text
							print(post_date_value)
					except Exception as e:
							print(f"Không lấy ngày đăng bài ở {index+1}")
							post_date_value = None
					
					####### Check for "see more" button and click it if present
					try:
							see_more_button = element.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl') and contains(@role, 'button') and contains(text(), 'Xem thêm')]")
							if see_more_button:
									see_more_button.click()
									time.sleep(1)  # Wait for content to expand
					except Exception as e:
							print("Post không có nút xem thêm")

					# Get the content
					content_element_1s = element.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")

					if len(content_element_1s) > 1:
							print('1')
							content_value = content_element_1s[1].text
					else: 
							time.sleep(1)
							print('2')
							content_element_2s = element.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u")

							if content_element_2s:
									content_value = content_element_2s[0].text
							else:
									content_value = None
									print("No content found for this post")
					
					# Get images
					image_elements = element.find_elements(By.TAG_NAME, 'img')
					image_urls = [img.get_attribute('src') for img in image_elements if int(img.get_attribute('width')) > 100 and int(img.get_attribute('height')) > 100]
					if image_urls:
							post_image_dir = os.path.join(image_save_dir, f'post_{index+1}')
							if not os.path.exists(post_image_dir):
									os.makedirs(post_image_dir)
							
							for i, image_url in enumerate(image_urls, start=1):
									image_path = os.path.join(post_image_dir, f'image_{i}.jpg')
									try:
											image_data = requests.get(image_url).content
											with open(image_path, 'wb') as img_file:
													img_file.write(image_data)
									except Exception as e:
											print(f"post {index+1} không có ảnh")
							image_value = ", ".join(image_urls)
					else:
							image_value = None

					# Append data to lists
					post_href.append(post_href_value)
					hr_id.append(hr_id_value)
					author_name.append(author_name_value)
					group_id.append(group_id_value)
					scrape_date.append(scrape_date_value)
					post_date.append(post_date_value)
					content.append(content_value)
					image.append(image_value)
					
					# Create DataFrame and save to CSV after each iteration
					df = pd.DataFrame({
							'post_href': post_href,
							'hr_id': hr_id,
							'author_name': author_name,
							'group_id': group_id,
							'scrape_date': scrape_date,
							'post_date': post_date,
							'content': content,
							'image': image
					})
					df.to_csv(csv_file_path, index=False, encoding='utf-8')
					print(f"Lưu csv post {index+1}")


Thứ Tư, 23 Tháng 10, 2024 lúc 18:00
2
Lưu csv post 1
Không lấy ngày đăng bài ở 2
1
Lưu csv post 2
Thứ Hai, 21 Tháng 10, 2024 lúc 18:05
2
Lưu csv post 3
Thứ Hai, 21 Tháng 10, 2024 lúc 21:28
1
Lưu csv post 4
Thứ Hai, 21 Tháng 10, 2024 lúc 21:19
Post không có nút xem thêm
1
Lưu csv post 5
Thứ Ba, 22 Tháng 10, 2024 lúc 15:25
Post không có nút xem thêm
1
Lưu csv post 6
Thứ Ba, 22 Tháng 10, 2024 lúc 15:24
2
Lưu csv post 7
Thứ Ba, 22 Tháng 10, 2024 lúc 10:50
2
Lưu csv post 8
Thứ Ba, 22 Tháng 10, 2024 lúc 10:49
2
Lưu csv post 9
Thứ Hai, 21 Tháng 10, 2024 lúc 22:10
1
Lưu csv post 10
Thứ Hai, 21 Tháng 10, 2024 lúc 23:56
2
Lưu csv post 11
Thứ Hai, 21 Tháng 10, 2024 lúc 16:21
Post không có nút xem thêm
2
Lưu csv post 12
Chủ Nhật, 20 Tháng 10, 2024 lúc 01:06
1
Lưu csv post 13
Thứ bảy, 19 Tháng 10, 2024 lúc 20:45
2
Lưu csv post 14
Thứ Sáu, 18 Tháng 10, 2024 lúc 11:54
2
Lưu csv post 15
Thứ Sáu, 11 Tháng 10, 2024 lúc 10:00
2
Lưu csv post 16
Thứ Sáu, 18 Tháng 10, 2024 lúc 11:15
1
Lưu csv post 17
Thứ Tư

KeyboardInterrupt: 

# lấy cookie

In [1]:
import os
import time
import re
import random
import requests
from datetime import datetime
import random
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import json
import csv

In [5]:
# Lấy danh sách cookie từ trình duyệt
cookies = driver.get_cookies()

# In cookie ra màn hình
print("Cookies:")
for cookie in cookies:
    print(cookie)

# Lưu cookie vào file JSON để sử dụng sau này
with open("facebook_cookies.json", "w") as file:
    json.dump(cookies, file, indent=4)

print("Cookies đã được lưu vào file facebook_cookies.json")

Cookies:
{'domain': '.facebook.com', 'expiry': 1752639183, 'httpOnly': True, 'name': 'fr', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '0tRjTwCU2U4F8hWLU.AWcoGjxymr1qHnuUE_7SJ4aypY1pIsPrg3WdIg7SnhVz1qynmUU.BoAH_p..AAA.0.0.BoAIAC.AWeJkiLh5mrBf61GzdCORM0aXu8'}
{'domain': '.facebook.com', 'expiry': 1776399182, 'httpOnly': True, 'name': 'xs', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '12%3AXIScaWSUJ_K5Wg%3A2%3A1744863231%3A-1%3A12159'}
{'domain': '.facebook.com', 'expiry': 1776399182, 'httpOnly': False, 'name': 'c_user', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '100075350970914'}
{'domain': '.facebook.com', 'expiry': 1745467986, 'httpOnly': False, 'name': 'dpr', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '1.5'}
{'domain': '.facebook.com', 'httpOnly': False, 'name': 'presence', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'C%7B%22t3%22%3A%5B%5D%2C%22utc3%22%3A1744863186555%2C%22v%22%3A1%7D'}
{'domain': '.facebook.com'

In [ ]:
import os
import time
import re
import random
import requests
from datetime import datetime
import random
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import json
import csv

# Function to read proxy config from file
def read_proxy_config(file_path):
    config = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            config[key] = value
    return config

# Read proxy settings from text file
proxy_config = read_proxy_config(r'C:\working\job_rcm\job_rcm_code\config.txt')
PROXY_HOST = proxy_config['PROXY_HOST']
PROXY_PORT = proxy_config['PROXY_PORT']
PROXY_USER = proxy_config['PROXY_USER']
PROXY_PASS = proxy_config['PROXY_PASS']

# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Proxy settings
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False, "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")

# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)

# Path lưu cookie
cookie_file = r"C:\working\job_rcm\job_rcm_code\job_scraping\facebook\facebook_cookies.json"

# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("http://www.facebook.com")

# Load cookies nếu có
try:
    with open(cookie_file, "r") as file:
        cookies = json.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    # Refresh để xem có đăng nhập thành công không
    driver.refresh()
    time.sleep(3)

    if "login" not in driver.current_url:
        print("Đăng nhập bằng cookie thành công!")
    else:
        print("Cookie hết hạn, đăng nhập lại...")
        raise Exception("Cookie Expired")
except Exception as e:
    print("Không thể đăng nhập bằng cookie:", str(e))
    
    # Đăng nhập bằng tài khoản & mật khẩu
    time.sleep(4)
    email = driver.find_element(By.NAME, "email")
    password = driver.find_element(By.NAME, "pass")
    email.send_keys("nguyenanhnguyen111666@gmail.com")
    password.send_keys("tranduyluan11062003")

    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

    # Chờ trang load hoàn tất
    time.sleep(5)

    # Kiểm tra nếu đăng nhập thành công, lưu cookie mới
    if "login" not in driver.current_url:
        print("Đăng nhập thành công! Lưu cookie mới...")
        
        # Lưu cookie vào file JSON
        cookies = driver.get_cookies()
        with open(cookie_file, "w") as file:
            json.dump(cookies, file, indent=4)
        
        print("Cookies đã được lưu vào", cookie_file)
    else:
        print("Đăng nhập thất bại! Vui lòng kiểm tra lại thông tin tài khoản.")
        
time.sleep(5)
# Open a new tab
# driver.execute_script("window.open('https://www.facebook.com/groups/160325109320298/');") 
driver.execute_script("window.open('https://www.facebook.com/groups/misdue/');") 

### cần làm nhiều group trong 

driver.switch_to.window(driver.window_handles[-1])
# Scale screen to 60% for the new tab as well
driver.execute_script("document.body.style.zoom='60%'")


C:\Users\trand\AppData\Local\Temp\ipykernel_17116\338910392.py:121: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


Đăng nhập bằng cookie thành công!


In [2]:
base_save_dir = r'C:\working\job_rcm\data\facebook'
csv_file_path = os.path.join(base_save_dir, 'post_hrefs.csv')

def read_post_hrefs_from_csv():
    if not os.path.exists(csv_file_path):
        return []
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        return [row[0] for row in reader]

# Function to save post href to the CSV file
def save_post_href_to_csv(post_href):
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([post_href])

group_href = "https://www.facebook.com/groups/3272112773116278/"

actions = ActionChains(driver)


post_href_list = read_post_hrefs_from_csv()

# Scraping loop
while True:
			feed_element = driver.find_element(By.XPATH, '//*[@role="feed"]')
			post_elements = driver.find_elements(By.XPATH, "//div[@class='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z']")
			# scrape_time = time.localtime()  # Record the scrape time

			for element in post_elements:
						scrape_time = time.localtime()  # Record the scrape time
						actions.move_to_element(element).perform()
						time.sleep(1)  

						# Get post href
						post_href_element = element.find_element(By.CSS_SELECTOR, 
								"div.html-div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1q0g3np a")
						post_href_value = post_href_element.get_attribute('href')
						if post_href_value in post_href_list:
								continue

						# Get author name
						try:
								author_name_element = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
								author_name_value = author_name_element[0].text.strip()
						except:
								author_name_value = None
						# Switch tabs and get HR ID
						actions.move_to_element(author_name_element[0]).key_down(Keys.CONTROL).click(author_name_element[0]).key_up(Keys.CONTROL).perform()
						time.sleep(2)
						current_tabs = driver.window_handles

						if len(current_tabs) > 2:
								driver.switch_to.window(current_tabs[-1])
								try :
										profile_elements = driver.find_elements(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx')
										hr_id_value = profile_elements[6].get_attribute('href')
								except:
										hr_id_value = None
								time.sleep(2)
								driver.close()
								driver.switch_to.window(current_tabs[1])
						else:
								print("Only one tab, no extra tab to close.")

						# Get post date
						try:
										hover_elements = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
										# time.sleep(5)
										actions.move_to_element(hover_elements[1]).perform()
										time.sleep(5)
										html =driver.page_source
										date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
										post_date_value = date_element.text
										print(post_date_value)
						except Exception as e:
										post_date_value = None

						# Check for "see more" button and click if present
						try:
								see_more_button = element.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl') and contains(@role, 'button') and contains(text(), 'Xem thêm')]")
								if see_more_button:
										see_more_button.click()
										time.sleep(1)
						except Exception as e:
								print("No 'see more' button found")

						# Get the content
						try :
								color_content_elements = element.find_elements(By.CSS_SELECTOR, "div.x6s0dn4.x78zum5.xdt5ytf.x5yr21d.xl56j7k.x10l6tqk.x17qophe.x13vifvy.xh8yej3")
								content_element_1s = element.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")

								if color_content_elements:  # Kiểm tra nếu tìm thấy phần tử
										color_content = color_content_elements[0]
										content_element_1s = color_content.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")
										
										if content_element_1s:
												print('1')
												content_value = content_element_1s[0].text
										else:
												content_value = None
								else:
										raise NoSuchElementException
								
						except NoSuchElementException:
								time.sleep(1)
								print('2')
								
								content_element_2s = element.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u")
								content_value = content_element_2s[0].text if content_element_2s else None
								

						# Get images
						image_elements = element.find_elements(By.TAG_NAME, 'img')
						image_urls = [img.get_attribute('src') for img in image_elements if int(img.get_attribute('width')) > 300 and int(img.get_attribute('height')) > 300]

						# Save post data and images
						post_data = {
								'post_href': post_href_value,
								'author_name': author_name_value,
								'hr_id': hr_id_value,
								'group_id': group_href,
								'scrape_date': time.strftime('%Y-%m-%d %H:%M:%S'),
								'post_date': post_date_value,
								'content': content_value,
								'images': image_urls
						}
						
						# save_post_data(post_data, element.get_attribute('outerHTML'), image_urls, scrape_time)
						# save_post_data(post_data,html, image_urls, scrape_time)

						# Append post_href_value to the CSV file
						save_post_href_to_csv(post_href_value)
						# Also append to the list for any further processing in the same run
						post_href_list.append(post_href_value)

						# Sleep before the next post to avoid rate limiting
						time.sleep(1)
						print(content_value)

			# Scroll down the page to load more posts
			driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
			time.sleep(5)


2
Uyen Uyen
Only one tab, no extra tab to close.
2
Ngọc Vy
2
 TUYỂN SINH THẠC SĨ NGÀNH HỆ THỐNG THÔNG TIN QUẢN LÝ - ĐỢT 1/2025 
Only one tab, no extra tab to close.


KeyboardInterrupt: 

In [133]:
import time
from selenium.common.exceptions import NoSuchElementException

try:
    color_content_elements = element.find_elements(By.CSS_SELECTOR, "div.x6s0dn4.x78zum5.xdt5ytf.x5yr21d.xl56j7k.x10l6tqk.x17qophe.x13vifvy.xh8yej3")
    
    if color_content_elements:  # Kiểm tra nếu tìm thấy phần tử
        color_content = color_content_elements[0]
        content_element_1s = color_content.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")
        
        if content_element_1s:
            print('1')
            content_value = content_element_1s[0].text
        else:
            content_value = None
    else:
        raise NoSuchElementException  # Giả lập lỗi để vào except

except NoSuchElementException:
    time.sleep(1)
    print('2')
    
    content_element_2s = element.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u")
    content_value = content_element_2s[0].text


2


In [150]:
feed_element = driver.find_element(By.XPATH, '//*[@role="feed"]')
post_elements = driver.find_elements(By.XPATH, "//div[@class='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z']")
element = post_elements[0]

In [148]:
post_elements

[<selenium.webdriver.remote.webelement.WebElement (session="d0a8ddf117a7f26a0fe6aa701b281783", element="f.B34DEF2FF611C20C081931B0A3C61D1D.d.FD64B9E629CC43C3756B3DDCCB7BDA78.e.292")>]

In [153]:
post_href_element = element.find_element(By.CSS_SELECTOR, 
								"div.html-div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1q0g3np a"
						)
post_href_value = post_href_element.get_attribute('href')
post_href_value

'https://www.facebook.com/groups/160325109320298/posts/543122581040547/?__cft__[0]=AZVwfB6aGg-pI00TT6Sk_jTqGZea4Mn_ipeUeo0zhq9fQuUaLU2_DMGi3wDuAB140fWup56PUixAL-8cLBRX57hYHLAfup07vhA1I5NhyTyptMbNfFIQGVwMeKGVyZikifp9AYRZT08b1M1xr1OXYIgnjhy5496EeV8q8cvl0z-nGPmIK1caF0nozuM9zU5WelcjdF9S1HLNKtdVVvK7gqoJ&__tn__=%2CO%2CP-R'

In [134]:
content_value

'Nộp đơn ở đâu chưa người đẹp?\nChưa thì nộp ở TC Data đi.\nTC Data tuyển dụng: Power Platform Developer… Xem thêm'

In [ ]:
from IPython.display import display
from PIL import Image
import requests
from io import BytesIO

# Lọc danh sách URL từ các ảnh có kích thước lớn hơn 300x300
img_urls = [
    img.get_attribute('src') for img in image_elements
    if int(img.get_attribute('width')) > 300 and int(img.get_attribute('height')) > 300
]

for img_url in img_urls:
    try:
        response = requests.get(img_url)  # Tải từng ảnh một
        response.raise_for_status()  # Kiểm tra nếu có lỗi HTTP
        image = Image.open(BytesIO(response.content))
        display(image)  # Hiển thị ảnh
    except Exception as e:
        print(f"Error loading image {img_url}: {e}")


# new code

#### import

In [1]:
import os
import time
import re
import random
import requests
from datetime import datetime
import random
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import json
import csv
from selenium.common.exceptions import NoSuchElementException

#### Cài đặt path

In [2]:
# ChromeDriver path
chrome_driver_path = 'C:/Users/trand/chromedriver.exe'

# Setup directory paths
base_save_dir = r'C:\working\job_rcm\data\facebook'
image_save_dir = os.path.join(base_save_dir, 'post_image')
csv_file_path = os.path.join(base_save_dir, 'post_hrefs.csv')
group_href_csv_path = os.path.join(base_save_dir, 'group_href.csv')

#### cài đặt và viết các hàm

In [15]:
# Đọc danh sách group từ CSV
def read_group_hrefs():
    groups = []
    if os.path.exists(group_href_csv_path):
        with open(group_href_csv_path, 'r', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)  # Bỏ qua tiêu đề
            for row in reader:
                if len(row) >= 1:
                    groups.append([row[0], row[1] if len(row) > 1 else ""])
    return groups

def update_last_scraped(group_href):
    groups = read_group_hrefs()
    for group in groups:
        if group[0] == group_href:
            group[1] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(group_href_csv_path, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["group_href", "last_scraped"])
        writer.writerows(groups)

group_hrefs = read_group_hrefs()
current_group_index = 0  # Biến lưu trạng thái group hiện tại

def get_next_group_href():
    global current_group_index
    if current_group_index < len(group_hrefs):
        group_href = group_hrefs[current_group_index][0]
        current_group_index += 1
        return group_href
    return None

# Create directory if it does not exist
if not os.path.exists(image_save_dir):
    os.makedirs(image_save_dir)

# Function to read post hrefs from the CSV file
def read_post_hrefs_from_csv():
    if not os.path.exists(csv_file_path):
        return []
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        return [row[0] for row in reader]

# Function to save post href to the CSV file
def save_post_href_to_csv(post_href):
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([post_href])

# Hàm chuyển đổi struct_time thành đối tượng datetime
def convert_scrape_time(scrape_time):
    return datetime.fromtimestamp(time.mktime(scrape_time))

# Hàm tạo thư mục theo thời gian scrape
def create_directory_structure(scrape_time):
    scrape_time = convert_scrape_time(scrape_time)  # Chuyển struct_time thành datetime
    year_dir = os.path.join(base_save_dir, scrape_time.strftime("%Y"))
    month_dir = os.path.join(year_dir, scrape_time.strftime("%m"))
    day_dir = os.path.join(month_dir, scrape_time.strftime("%d"))
    hour_dir = os.path.join(day_dir, scrape_time.strftime("%H"))

    for i in range(1, 100):  # Tối đa 100 phần
        part_dir = os.path.join(hour_dir, f'part_{i}')
        if not os.path.exists(part_dir):
            os.makedirs(part_dir)
            return part_dir

        if len(os.listdir(part_dir)) < 300:  # Mỗi thư mục chứa tối đa 300 post
            return part_dir

# Hàm lưu dữ liệu post
def save_post_data(post_data, post_html, images, scrape_time):
    part_dir = create_directory_structure(scrape_time)

    scrape_time_str = convert_scrape_time(scrape_time).strftime('%Y-%m-%d_%H%M%S')  # Thêm giây để đảm bảo tên duy nhất
    detail_folder_name = f"{scrape_time_str}_group_{post_data['group_id'].split('/')[-2]}"
    detail_folder = os.path.join(part_dir, detail_folder_name)
    os.makedirs(detail_folder, exist_ok=True)

    # Lưu file JSON
    json_path = os.path.join(detail_folder, 'data.json')
    with open(json_path, 'w', encoding='utf-8') as json_file:
        json.dump(post_data, json_file, ensure_ascii=False, indent=4)

    # Lưu file HTML
    html_path = os.path.join(detail_folder, 'html.txt')
    with open(html_path, 'w', encoding='utf-8') as html_file:
        html_file.write(post_html)

    # Lưu hình ảnh
    image_folder = os.path.join(detail_folder, 'images')
    os.makedirs(image_folder, exist_ok=True)
    for idx, img_url in enumerate(images):
        img_path = os.path.join(image_folder, f'image_{idx+1}.jpg')
        try:
            img_data = requests.get(img_url).content
            with open(img_path, 'wb') as img_file:
                img_file.write(img_data)
        except Exception as e:
            print(f"Error downloading image: {e}")

################# Set up Chrome options and driver #################

def read_config(file_path):
    config = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config



# Read proxy settings from text file
proxy_config = read_config(r'C:\working\job_rcm\job_rcm_code\config.txt')

PROXY_HOST = proxy_config.get('PROXY_HOST', '')
PROXY_PORT = proxy_config.get('PROXY_PORT', '')
PROXY_USER = proxy_config.get('PROXY_USER', '')
PROXY_PASS = proxy_config.get('PROXY_PASS', '')

# Proxy settings
manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)

# Set up Chrome options
options = Options()
options.add_argument("--disable-infobars")
prefs = {"credentials_enable_service": False,
     "profile.password_manager_enabled": False}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

# Randomize User-Agent
user_agents = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
	
# Set up proxy
pluginfile = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)
options.add_extension(pluginfile)

# Path lưu cookie
cookie_file = r"C:\working\job_rcm\job_rcm_code\job_scraping\facebook\facebook_cookies.json"




def scrape_post(driver, element, actions, post_href_list, group_href):
			"""Hàm để lấy thông tin bài viết từ một phần tử web"""
			
			scrape_time = time.localtime()  # Ghi lại thời gian scrape
			actions.move_to_element(element).perform()
			time.sleep(1)  

			# Lấy post href
			try:
					post_href_element = element.find_element(By.CSS_SELECTOR, 
													"div.html-div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1q0g3np a")
					post_href_value = post_href_element.get_attribute('href')
					if post_href_value in post_href_list:
							return None
			except NoSuchElementException:
					post_href_value = None

			# Lấy tên tác giả
			try:
					author_name_element = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
					author_name_value = author_name_element[0].text.strip()
			except:
					author_name_value = None

			# Chuyển tab và lấy HR ID
			actions.move_to_element(author_name_element[0]).key_down(Keys.CONTROL).click(author_name_element[0]).key_up(Keys.CONTROL).perform()
			time.sleep(2)
			current_tabs = driver.window_handles
			
			if len(current_tabs) > 2:
					driver.switch_to.window(current_tabs[-1])
					try :
									profile_elements = driver.find_elements(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx')
									hr_id_value = profile_elements[6].get_attribute('href')
					except:
									hr_id_value = None
					time.sleep(2)
					driver.close()
					driver.switch_to.window(current_tabs[1])
			else:
					print("Only one tab, no extra tab to close.")

			# Lấy ngày đăng bài
			try:
					hover_elements = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
					actions.move_to_element(hover_elements[1]).perform()

					time.sleep(5)

					date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
					post_date_value = date_element.text
					print(post_date_value)
			except:
					post_date_value = None

			# Kiểm tra và bấm "Xem thêm" nếu có
			try:
					see_more_button = element.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl') and contains(@role, 'button') and contains(text(), 'Xem thêm')]")
					if see_more_button:
							see_more_button.click()
							time.sleep(1)
			except:
					print("No 'see more' button found")

			# Lấy nội dung bài viết
			content_value = None
			try:
					color_content_elements = element.find_elements(By.CSS_SELECTOR, "div.x6s0dn4.x78zum5.xdt5ytf.x5yr21d.xl56j7k.x10l6tqk.x17qophe.x13vifvy.xh8yej3")

					if color_content_elements:
							color_content = color_content_elements[0]
							content_element_1s = color_content.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")

							content_value = content_element_1s[0].text if content_element_1s else None
					else:
							raise NoSuchElementException
					
			except NoSuchElementException:
					try:
							content_element_2s = element.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u")
							content_value = content_element_2s[0].text if content_element_2s else None
					except:
							content_value = None

			# Lấy ảnh
			image_elements = element.find_elements(By.TAG_NAME, 'img')
			image_urls = [img.get_attribute('src') for img in image_elements if img.get_attribute('width') and img.get_attribute('height') and int(img.get_attribute('width')) > 300 and int(img.get_attribute('height')) > 300]

			# Lưu dữ liệu bài viết
			post_data = {
					'post_href': post_href_value,
					'author_name': author_name_value,
					'hr_id': hr_id_value,
					'group_id': group_href,
					'scrape_date': time.strftime('%Y-%m-%d %H:%M:%S'),
					'post_date': post_date_value,
					'content': content_value,
					'images': image_urls
			}

			# save_post_data(post_data,html, image_urls, scrape_time)
			save_post_href_to_csv(post_href_value)
			post_href_list.append(post_href_value)

			return post_data

#### khởi chạy drive và đăng nhập

In [16]:
# Launch browser
driver = webdriver.Chrome(chrome_driver_path, options=options)
driver.get("http://www.facebook.com")

# Load cookies nếu có	
try:
    with open(cookie_file, "r") as file:
        cookies = json.load(file)
        for cookie in cookies:
            driver.add_cookie(cookie)

    # Refresh để xem có đăng nhập thành công không
    driver.refresh()
    time.sleep(3)

    if "login" not in driver.current_url:
        print("Đăng nhập bằng cookie thành công!")
    else:
        print("Cookie hết hạn, đăng nhập lại...")
        raise Exception("Cookie Expired")
except Exception as e:
    print("Không thể đăng nhập bằng cookie:", str(e))
    
    # Đăng nhập bằng tài khoản & mật khẩu
    time.sleep(4)
    email = driver.find_element(By.NAME, "email")
    password = driver.find_element(By.NAME, "pass")
    email.send_keys("nguyenanhnguyen111666@gmail.com")
    password.send_keys("tranduyluan11062003")

    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

    # Chờ trang load hoàn tất
    time.sleep(5)

    # Kiểm tra nếu đăng nhập thành công, lưu cookie mới
    if "login" not in driver.current_url:
        print("Đăng nhập thành công! Lưu cookie mới...")
        
        # Lưu cookie vào file JSON
        cookies = driver.get_cookies()
        with open(cookie_file, "w") as file:
            json.dump(cookies, file, indent=4)
        
        print("Cookies đã được lưu vào", cookie_file)
    else:
        print("Đăng nhập thất bại! Vui lòng kiểm tra lại thông tin tài khoản.")

time.sleep(5)

C:\Users\trand\AppData\Local\Temp\ipykernel_18728\93514452.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options)


Đăng nhập bằng cookie thành công!


In [ ]:
# Duyệt qua các nhóm
group_href = get_next_group_href()
post_href_list = read_post_hrefs_from_csv()

while group_href:
    print(f"Đang truy cập nhóm: {group_href}")
    driver.execute_script(f"window.open('{group_href}');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.execute_script("document.body.style.zoom='60%'")

    post_count = 0  # Reset post count mỗi lần đổi nhóm
    actions = ActionChains(driver)

    while post_count < 5:  # Số lượng bài cần thu thập trước khi đổi nhóm
        try:
            feed_element = driver.find_element(By.XPATH, '//*[@role="feed"]')
            post_elements = driver.find_elements(By.XPATH, "//div[@class='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z']")
            for element in post_elements:
                
                post_data = scrape_post(driver, element, actions, post_href_list, group_href)
                print(post_data)
                
								
                time.sleep(1)
                post_count += 1
                print(f"Đã thu thập {post_count} bài trong nhóm {group_href}")
                if post_count >= 5:
                    break
        except Exception as e:
            print(f"Lỗi khi thu thập bài viết: {e}")
        
        # Cuộn xuống để tải thêm bài
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    
    print(f"Hoàn thành nhóm {group_href}, cập nhật last_scraped và chuyển sang nhóm tiếp theo...")
    update_last_scraped(group_href)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    group_href = get_next_group_href()

print("Không còn nhóm nào để thu thập, dừng chương trình.")
driver.quit()


Đang truy cập nhóm: https://www.facebook.com/groups/160325109320298/
{'post_href': 'https://www.facebook.com/groups/160325109320298/posts/543122581040547/?__cft__[0]=AZUR_ZqSBzitCqjLr6SMZMtmw4NwvQ7wmg-5aNUMx2-4prQTfU0lYdjpGveXKb5epA66uVLv8rVDbFRql7abtc0GrHaChggt4yB3h1w-N_JbLFRXn3TnCyVWbD3RLUT_iJgqpq9f1NWykuTZoPo1B2pe_kPMXFch43YwOV6DT-bNX8TK1BTW2wc76Hf1f8npG9yvDk-fR0v5M5TYZ5c1Sy-b&__tn__=%2CO%2CP-R', 'author_name': 'Phương Uyên', 'hr_id': 'https://www.facebook.com/profile.php?id=100003871125916', 'group_id': 'https://www.facebook.com/groups/160325109320298/', 'scrape_date': '2025-03-12 14:16:18', 'post_date': None, 'content': None, 'images': ['https://scontent-sin2-2.xx.fbcdn.net/v/t39.30808-6/337171979_749691466868916_2917163268654261696_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=aa7b47&_nc_ohc=zcYVkNdG0LcQ7kNvgHv-B4Y&_nc_oc=AdgDIuDXfXr54CivVCwJ_d2N_AYeWZYHIUcmXJTeaCcrNu_Eue4kKl0ZL8ndnDwQs1xa6cuNT1noBzBa6COkSme2&_nc_zt=23&_nc_ht=scontent-sin2-2.xx&_nc_gid=AACGMUQOe2R6qoCmrf37xjy&oh=00_AYFb3mzsJ

In [ ]:



# Open a new tab
# driver.execute_script("window.open('https://www.facebook.com/groups/160325109320298/');") 

# driver.execute_script(f"window.open('{group_href}');")

# driver.switch_to.window(driver.window_handles[-1])
# # Scale screen to 60% for the new tab as well
# driver.execute_script("document.body.style.zoom='60%'")

group_href = get_next_group_href()

#################################
# Scraping data #################
#################################

actions = ActionChains(driver)
post_href_list = read_post_hrefs_from_csv()

post_count = 0 
n = 1


# Scraping loop
while True:
        feed_element = driver.find_element(By.XPATH, '//*[@role="feed"]')
        post_elements = driver.find_elements(By.XPATH, "//div[@class='x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z']")
        # scrape_time = time.localtime()  # Record the scrape time

        for element in post_elements:
                
                if post_count >= n:
                        print(f"Đã thu thập đủ {n} bài trong nhóm {group_href}, chuyển sang nhóm tiếp theo...")

                        driver.close()

                        # Lấy nhóm mới
                        group_href = get_group_href()
                        
                        if not group_href:
                            print("Không có nhóm nào để tiếp tục, dừng chương trình.")
                            driver.quit()
                            exit()
                        # Mở tab mới với nhóm mới
                        driver.execute_script(f"window.open('{group_href}');")
                        driver.switch_to.window(driver.window_handles[-1])
                        driver.execute_script("document.body.style.zoom='60%'")
                        post_count = 0
                        continue
                scrape_time = time.localtime()  # Record the scrape time
                actions.move_to_element(element).perform()
                time.sleep(1)  

                # Get post href
                post_href_element = element.find_element(By.CSS_SELECTOR, 
                        "div.html-div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1q0g3np a")
                post_href_value = post_href_element.get_attribute('href')
                if post_href_value in post_href_list:
                        continue

                # Get author name
                try:
                        author_name_element = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
                        author_name_value = author_name_element[0].text.strip()
                except:
                        author_name_value = None
                # Switch tabs and get HR ID
                actions.move_to_element(author_name_element[0]).key_down(Keys.CONTROL).click(author_name_element[0]).key_up(Keys.CONTROL).perform()
                time.sleep(2)
                current_tabs = driver.window_handles

                if len(current_tabs) > 2:
                        driver.switch_to.window(current_tabs[-1])
                        try :
                                profile_elements = driver.find_elements(By.CSS_SELECTOR, 'a.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx')
                                hr_id_value = profile_elements[6].get_attribute('href')
                        except:
                                hr_id_value = None
                        time.sleep(2)
                        driver.close()
                        driver.switch_to.window(current_tabs[1])
                else:
                        print("Only one tab, no extra tab to close.")

                # Get post date
                try:
                                hover_elements = element.find_elements(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs')
                                # time.sleep(5)
                                actions.move_to_element(hover_elements[1]).perform()
                                time.sleep(5)
                                html =driver.page_source
                                date_element = driver.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1nxh6w3.x1sibtaa.xo1l8bm.xzsf02u")
                                post_date_value = date_element.text
                                print(post_date_value)
                except Exception as e:
                                post_date_value = None

                # Check for "see more" button and click if present
                try:
                        see_more_button = element.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl') and contains(@role, 'button') and contains(text(), 'Xem thêm')]")
                        if see_more_button:
                                see_more_button.click()
                                time.sleep(1)
                except Exception as e:
                        print("No 'see more' button found")

                # Get the content
                try :
                        color_content_elements = element.find_elements(By.CSS_SELECTOR, "div.x6s0dn4.x78zum5.xdt5ytf.x5yr21d.xl56j7k.x10l6tqk.x17qophe.x13vifvy.xh8yej3")
                        content_element_1s = element.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")

                        if color_content_elements:  # Kiểm tra nếu tìm thấy phần tử
                                color_content = color_content_elements[0]
                                content_element_1s = color_content.find_elements(By.CSS_SELECTOR, "div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs")
                                
                                if content_element_1s:
                                        print('1')
                                        content_value = content_element_1s[0].text
                                else:
                                        content_value = None
                        else:
                                raise NoSuchElementException
                        
                except NoSuchElementException:
                        time.sleep(1)
                        print('2')
                        
                        content_element_2s = element.find_elements(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u")
                        content_value = content_element_2s[0].text if content_element_2s else None
                        

                # Get images
                image_elements = element.find_elements(By.TAG_NAME, 'img')
                image_urls = [img.get_attribute('src') for img in image_elements if int(img.get_attribute('width')) > 300 and int(img.get_attribute('height')) > 300]

                # Save post data and images
                post_data = {
                        'post_href': post_href_value,
                        'author_name': author_name_value,
                        'hr_id': hr_id_value,
                        'group_id': group_href,
                        'scrape_date': time.strftime('%Y-%m-%d %H:%M:%S'),
                        'post_date': post_date_value,
                        'content': content_value,
                        'images': image_urls
                }
                
                # save_post_data(post_data, element.get_attribute('outerHTML'), image_urls, scrape_time)
                # save_post_data(post_data,html, image_urls, scrape_time)

                # Append post_href_value to the CSV file
                save_post_href_to_csv(post_href_value)
                # Also append to the list for any further processing in the same run
                post_href_list.append(post_href_value)
                print(f"đang thao tác ở post {post_count}")
                post_count += 1

                # Sleep before the next post to avoid rate limiting
                time.sleep(1)
                

        # Scroll down the page to load more posts
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
